## Exercise 2 - Naive Bayes

First name: Brian
<br>
Last name: Schweigler
<br>
Matriculation number: 16-102-071

#### (1a) In the correlation visualization, select the two features that have the most significant correlation to the target feature, Survived.

First some imports and preprocessing

In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing
import scipy
from mlxtend.classifier import OneRClassifier
from mlxtend.evaluate import accuracy_score

df = pd.read_csv("data/titanic.csv", index_col='Name')
# df.describe(include='all')
pd.set_option('display.max_colwidth', None)
print(df.head(5))
le = preprocessing.LabelEncoder()
#df["Name"] = le.fit_transform(df["Name"])
df["Survived"] = le.fit_transform(df["Survived"])
df["Sex"] = le.fit_transform(df["Sex"])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
                                                    Survived  Pclass     Sex  \
Name                                                                           
Mr. Owen Harris Braund                                     0       3    male   
Mrs. John Bradley (Florence Briggs Thayer) Cumings         1       1  female   
Miss. Laina Heikkinen                                      1       3  female   
Mrs. Jacques Heath (Lily May Peel) Futrelle                1       1  female   
Mr. William Henry Allen                                    0       3    male   

                                                     Age  \
Name                                                       
Mr. Owen Harris Braund                              22.0   
Mrs. John Bradley (Florence Briggs Thayer) Cumings  38.0   
Miss. Laina Heikkinen                               26.0   
Mrs. Jacques Heath (Lily May Peel) Futrelle        

In [5]:
correlation = df.corr()["Survived"]
correlation = correlation.apply(lambda entry: abs(entry))

print("Correlation of columns to column survived")
print(correlation.sort_values(ascending=False))


correlation.pop(correlation.idxmax())
temp_correlation = correlation
best = correlation.idxmax()
temp_correlation.pop(best)
second_best = temp_correlation.idxmax()

print(best + " has the highest correlation, " + second_best + " the second highest")

Correlation of columns to column survived
Survived                   1.000000
Sex                        0.542152
Pclass                     0.336528
Fare                       0.256179
Parents/Children Aboard    0.080097
Age                        0.059665
Siblings/Spouses Aboard    0.037082
Name: Survived, dtype: float64
Sex has the highest correlation, Pclass the second highest
Accuracy if assuming everyone survived : 0.3855693348365276
Accuracy if assuming everyone died : 0.6144306651634723


#### (1b) Using Naive Bayes classifier and the most two significant features to predict the Survival of the travellers.

In [11]:
naive_bayes = GaussianNB()
x_train, x_test, y_train, y_test = train_test_split(df[best, second_best], df["Survived"], test_size=0.4, random_state=6)
naive_bayes.fit(x_train, y_train)
test_predictions = naive_bayes.predict(x_test)
accuracy = accuracy_score(y_test, test_predictions)
print(f'Accuracy: {round(accuracy,4)} for Naive Bayes based on the two best correlation values.' )

#Select the two highest correlated predictors. and the targe value

#Now lets train and evaluate the model based on Sex and class:



# survived = 0
# died = 0
# for index, row in df.iterrows():
#     if row["Survived"] == 1:
#         survived += 1
#     else:
#         died += 1
#
# print("Accuracy if assuming everyone survived :", survived/(survived+died))
# print("Accuracy if assuming everyone died :", died/(survived+died))

Traceback (most recent call last):
  File "C:\Program Files\JetBrains\PyCharm 2021.1\plugins\python\helpers-pro\jupyter_debug\pydev_jupyter_utils.py", line 64, in attach_to_debugger
    debugger.prepare_to_run(enable_tracing_from_start=False)
TypeError: PyDB.prepare_to_run() got an unexpected keyword argument 'enable_tracing_from_start'
Failed to connect to target debugger.


KeyError: ('Sex', 'Pclass')

In general, "choose most frequent to be the default rule".
In this case, assuming that everyone died is the best default rule, and we should aim to beat with our prediction approaches.


#### (1b) What is the best 1R for this dataset?
Very likely, without prior information, we can assume that due to "mothers and children first" when the titanic sank,
that those are most likely to have survived, thus gender (Sex) or if they are a parent/child.

So let's test this:

In [ ]:
X_d = df[["Sex"]]
y = df["Survived"]
Xd_train, Xd_test, y_train, y_test = train_test_split(X_d, y, test_size=0.3, random_state=2)
oner = OneRClassifier()
oner.fit(Xd_train.to_numpy(), y_train)
y_pred = oner.predict(Xd_test.to_numpy())

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy using outdated binary gender: ", accuracy)

X_d_fam = df[["Parents/Children Aboard"]]
y_fam = df["Survived"]
Xd_train_fam, Xd_test_fam, y_train_fam, y_test_fam = train_test_split(X_d_fam, y_fam, test_size=0.3, random_state=2)
oner_fam = OneRClassifier()
oner_fam.fit(Xd_train_fam.to_numpy(), y_train_fam)
y_pred_fam = oner_fam.predict(Xd_test_fam.to_numpy())

accuracy_fam = accuracy_score(y_test_fam, y_pred_fam)
print("Parents/Children accuracy: ", accuracy_fam)

Thus the outdated binary gender within this dataset (M/F) is the best 1R.

#### (1c) Can you produce a second rule based on a single attribute with a good effectiveness?
For this we can simply look at all the variants:

In [ ]:
X_d_fare = df[["Fare"]]
y_fare = df["Survived"]
Xd_train_fare, Xd_test_fare, y_train_fare, y_test_fare = train_test_split(X_d_fare, y_fare, test_size=0.3, random_state=2)
oner_fare = OneRClassifier()
oner_fare.fit(Xd_train_fare.to_numpy(), y_train_fare)
y_pred_fare = oner_fare.predict(Xd_test_fare.to_numpy())
accuracy_fare = accuracy_score(y_test_fare, y_pred_fare)
print("Fare Accuracy: ", accuracy_fare)

X_d_pclass = df[["Pclass"]]
y_pclass = df["Survived"]
Xd_train_pclass, Xd_test_pclass, y_train_pclass, y_test_pclass = train_test_split(X_d_pclass, y_pclass, test_size=0.3, random_state=2)
oner_pclass = OneRClassifier()
oner_pclass.fit(Xd_train_pclass.to_numpy(), y_train_pclass)
y_pred_pclass = oner_pclass.predict(Xd_test_pclass.to_numpy())

accuracy_pclass = accuracy_score(y_test_pclass, y_pred_pclass)
print("Pclass Accuracy: ", accuracy_pclass)

X_d_age = df[["Age"]]
y_age = df["Survived"]
Xd_train_age, Xd_test_age, y_train_age, y_test_age = train_test_split(X_d_age, y_age, test_size=0.3, random_state=2)
oner_age = OneRClassifier()
oner_age.fit(Xd_train_age.to_numpy(), y_train_age)
y_pred_age = oner_age.predict(Xd_test_age.to_numpy())

accuracy_age = accuracy_score(y_test_age, y_pred_age)
print("Age accuracy: ", accuracy_age)

X_d_sib = df[["Siblings/Spouses Aboard"]]
y_sib = df["Survived"]
Xd_train_sib, Xd_test_sib, y_train_sib, y_test_sib = train_test_split(X_d_sib, y_sib, test_size=0.3, random_state=2)
oner_sib = OneRClassifier()
oner_sib.fit(Xd_train_sib.to_numpy(), y_train_sib)
y_pred_sib = oner_sib.predict(Xd_test_sib.to_numpy())

accuracy_sib = accuracy_score(y_test_sib, y_pred_sib)
print("Sibling/Spouse accuracy: ", accuracy_sib)



Thus, we can conclude that using the Pclass (or Fare which correlates with it) are the best ones.
This is likely due to the fact that those who payed more (or are in a better class) are higher up on the ship, farther away from the engines.
Thus they had more time to escape.

-------------------------------

#### (2) Use a stock / market index for daily return of the day
First we simply load the dataset

In [ ]:
pd.set_option('display.max_colwidth', None)
stock_df = pd.read_csv("data/Nasdaq.csv", index_col='Date')
print(stock_df.head(10))
# stock_df['Date'] = stock_df['Date'].apply(pd.to_datetime)
stock_df.describe(include='all')

Look at daily return histograms:

In [ ]:
daily_return = np.empty(stock_df['Close'].shape)
#  From Slides: Daily return (r): Difference in percentage between the price at time t+1 and at time t
daily_return[0] = float('NaN') # The first
daily_return[1:] = np.ediff1d(stock_df['Close']) / stock_df['Close'][:-1]
stock_df.insert(loc=len(stock_df.columns), column='Daily Return', value=daily_return)

bins = int(len(daily_return) / 32)
plt.hist(daily_return, bins=bins)
plt.show()

An approach to compare binary and ternary solution.

In [ ]:
binary = (daily_return > 0).astype(float)
stock_df.insert(loc=len(stock_df.columns), column='Binary Decision', value=binary)

limit = 0.001
ternary = np.zeros(daily_return.shape)
ternary[np.where(daily_return > limit)] = 2
ternary[np.where(daily_return < limit)] = 1
stock_df.insert(loc=len(stock_df.columns), column='Ternary Decision', value=ternary)

print(stock_df.head(20))

Create 1R model using volume:

In [ ]:
X_d_stock = stock_df[["Volume"]]
y_stock = stock_df["Binary Decision"]
print("Length of DF: ", stock_df.shape[0])
test_size = 100/stock_df.shape[0]
print("Percentage of test_size to use last 100 days: ", test_size)
Xd_train_stock, Xd_test_stock, y_train_stock, y_test_stock = train_test_split(X_d_stock, y_stock, test_size=test_size, random_state=2, shuffle=False)
oner_stock = OneRClassifier()
oner_stock.fit(Xd_train_stock.to_numpy(), y_train_stock)
y_pred_stock = oner_stock.predict(Xd_test_stock.to_numpy())
accuracy_stock_bi = accuracy_score(y_test_stock, y_pred_stock)
print("Binary Accuracy: ", accuracy_stock_bi)

# As ternary does not play nicely with OneR, using binary only
# X_d_stock_ter = stock_df[["Volume"]]
# y_stock_ter = stock_df["Ternary Decision"]
# Xd_train_stock_ter, Xd_test_stock_ter, y_train_stock_ter, y_test_stock_ter = train_test_split(X_d_stock_ter, y_stock_ter, test_size=0.3)
# oner_stock_ter = OneRClassifier()
# oner_stock_ter.fit(Xd_train_stock_ter.to_numpy(), y_train_stock_ter)
# y_pred_stock_ter = oner_stock_ter.predict(Xd_test_stock_ter.to_numpy())
# accuracy_stock_ter = accuracy_score(y_test_stock_ter, y_pred_stock_ter)
# print("Ternary Accuracy: ", accuracy_stock_ter)

Create 1R model using volume and rolling averages:

In [ ]:
stock_df['Rolling Mean 5'] = stock_df['Close'].rolling(5).mean()
stock_df['Rolling Mean 10'] = stock_df['Close'].rolling(10).mean()
stock_df['Rolling Mean 20'] = stock_df['Close'].rolling(20).mean()
stock_df['Rolling Mean 50'] = stock_df['Close'].rolling(50).mean()
stock_df['Rolling Mean 200'] = stock_df['Close'].rolling(200).mean()


In [ ]:
X_d_stock_5 = stock_df[["Volume", "Rolling Mean 5"]]
y_stock_5 = stock_df["Binary Decision"]
test_size = 100/stock_df.shape[0]
Xd_train_stock_5, Xd_test_stock_5, y_train_stock_5, y_test_stock_5 = train_test_split(X_d_stock_5, y_stock_5, test_size=test_size, random_state=2, shuffle=False)
oner_stock_5 = OneRClassifier()
oner_stock_5.fit(Xd_train_stock_5.to_numpy(), y_train_stock_5)
y_pred_stock_5 = oner_stock_5.predict(Xd_test_stock_5.to_numpy())
accuracy_stock_bi_5 = accuracy_score(y_test_stock_5, y_pred_stock_5)
print("Rolling Mean 5 Accuracy: ", accuracy_stock_bi_5)

X_d_stock_10 = stock_df[["Volume", "Rolling Mean 10"]]
y_stock_10 = stock_df["Binary Decision"]
Xd_train_stock_10, Xd_test_stock_10, y_train_stock_10, y_test_stock_10 = train_test_split(X_d_stock_10, y_stock_10, test_size=test_size, random_state=2, shuffle=False)
oner_stock_10 = OneRClassifier()
oner_stock_10.fit(Xd_train_stock_10.to_numpy(), y_train_stock_10)
y_pred_stock_10 = oner_stock.predict(Xd_test_stock_10.to_numpy())
accuracy_stock_bi_10 = accuracy_score(y_test_stock_10, y_pred_stock_10)
print("Rolling Mean 10 Accuracy: ", accuracy_stock_bi_10)

X_d_stock_20 = stock_df[["Volume", "Rolling Mean 20"]]
y_stock_20 = stock_df["Binary Decision"]
Xd_train_stock_20, Xd_test_stock_20, y_train_stock_20, y_test_stock_20 = train_test_split(X_d_stock_20, y_stock_20, test_size=test_size, random_state=2, shuffle=False)
oner_stock_20 = OneRClassifier()
oner_stock_20.fit(Xd_train_stock_20.to_numpy(), y_train_stock_20)
y_pred_stock_20 = oner_stock_20.predict(Xd_test_stock_20.to_numpy())
accuracy_stock_bi_20 = accuracy_score(y_test_stock_20, y_pred_stock_20)
print("Rolling Mean 20 Accuracy: ", accuracy_stock_bi_20)

X_d_stock_50 = stock_df[["Volume", "Rolling Mean 50"]]
y_stock_50 = stock_df["Binary Decision"]
Xd_train_stock_50, Xd_test_stock_50, y_train_stock_50, y_test_stock_50 = train_test_split(X_d_stock_50, y_stock_50, test_size=test_size, random_state=2, shuffle=False)
oner_stock_50 = OneRClassifier()
oner_stock_50.fit(Xd_train_stock_50.to_numpy(), y_train_stock_50)
y_pred_stock_50 = oner_stock_50.predict(Xd_test_stock_50.to_numpy())
accuracy_stock_bi_50 = accuracy_score(y_test_stock_50, y_pred_stock_50)
print("Rolling Mean 50 Accuracy: ", accuracy_stock_bi_50)

X_d_stock_200 = stock_df[["Volume", "Rolling Mean 200"]]
y_stock_200 = stock_df["Binary Decision"]
Xd_train_stock_200, Xd_test_stock_200, y_train_stock_200, y_test_stock_200 = train_test_split(X_d_stock_200, y_stock_200, test_size=test_size, random_state=2, shuffle=False)
oner_stock_200 = OneRClassifier()
oner_stock_200.fit(Xd_train_stock_200.to_numpy(), y_train_stock_200)
y_pred_stock_200 = oner_stock_200.predict(Xd_test_stock_200.to_numpy())
accuracy_stock_bi_200 = accuracy_score(y_test_stock_200, y_pred_stock_200)
print("Rolling Mean 200 Accuracy: ", accuracy_stock_bi_200)


Weirdly enough, no matter what the rolling average, we seem to receive the same results.